In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds

In [0]:
dataset, info = tfds.load('fashion_mnist', as_supervised=True, split = ('train[:60%]', 'train[60%:]', 'test'),
        # tfds.Split.TRAIN.subsplit(tfds.percent[:80]),
        # tfds.Split.TRAIN.subsplit(tfds.percent[80:90]),
        # tfds.Split.TRAIN.subsplit(tfds.percent[90:]),
    with_info = True)

In [57]:
dataset

(<DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 <DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 <DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>)

In [0]:
train_dataset = dataset[0]
test_dataset = dataset[2]
valid_dataset = dataset[1]

In [59]:
for image,label in valid_dataset.batch(16).take(2):
    print(type(image))
    print(type(label))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [60]:
info

tfds.core.DatasetInfo(
    name='fashion_mnist',
    version=3.0.0,
    description='Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.',
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    = {Han Xiao and
                   Kashif Rasul and
                   Roland Vollgraf},
      title     = {Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning
                   Algorithms},
      journal   = {CoRR},
      volume

In [0]:
def preprocess(image,label):
    #to grayscale
    image = tf.dtypes.cast(image, tf.float32)
    label = tf.dtypes.cast(label, tf.float32)
    
    #resize
    image = tf.image.per_image_standardization(image)
    
    return image,label

In [0]:
train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)
valid_dataset = valid_dataset.map(preprocess)

In [0]:
class CustomNet(tf.keras.Model):
    def __init__(self):
        super(CustomNet, self).__init__()
        initializer = tf.initializers.GlorotUniform(seed=48)
        # Conv1
        self.wc1 = tf.Variable(initializer([3, 3, 1, 32]), trainable=True, name='wc1')
        
        # Conv2
        self.wc2 = tf.Variable(initializer([3, 3, 32, 64]), trainable=True, name='wc2')
        
         # Conv3
        self.wc3 = tf.Variable(initializer([3, 3, 64, 128]), trainable=True, name='wc2')
        
        # Flatten
        
        # Dense
        self.wd4 = tf.Variable(initializer([15488, 128]), trainable=True)    
        
        # Dense
        self.wd5 = tf.Variable(initializer([128, 10]), trainable=True)    
        
        self.bc1 = tf.Variable(tf.zeros([32]), dtype=tf.float32, trainable=True)
        self.bc2 = tf.Variable(tf.zeros([64]), dtype=tf.float32, trainable=True)
        self.bc3 = tf.Variable(tf.zeros([128]), dtype=tf.float32, trainable=True)     
        self.bd4 = tf.Variable(tf.zeros([128]), dtype=tf.float32, trainable=True)   
        self.bd5 = tf.Variable(tf.zeros([10]), dtype=tf.float32, trainable=True)   
    
    def call(self, x):
        # X = NHWC 
        # Conv1 + maxpool 2
        x = tf.nn.conv2d(x, self.wc1, strides=[1, 1, 1, 1], padding="VALID")
        x = tf.nn.bias_add(x, self.bc1)
        x = tf.nn.relu(x)
#         x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
        
        # Conv2 + maxpool 2
        x = tf.nn.conv2d(x, self.wc2, strides=[1, 1, 1, 1], padding="VALID")
        x = tf.nn.bias_add(x, self.bc2)
        x = tf.nn.relu(x)
#         x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
        
         # Conv3 + maxpool 3
        x = tf.nn.conv2d(x, self.wc3, strides=[1, 1, 1, 1], padding="VALID")
        x = tf.nn.bias_add(x, self.bc3)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
        
        # Flattten out
        # N X Number of Nodes
        # Flatten()
        x = tf.reshape(x, (tf.shape(x)[0], -1))
        
        # Dense1
        x = tf.matmul(x, self.wd4)
        x = tf.nn.bias_add(x, self.bd4)
        x = tf.nn.relu(x)

        
        # Dense2
        x = tf.matmul(x, self.wd5)
        x = tf.nn.bias_add(x, self.bd5)
        x = tf.nn.sigmoid(x)
        
        return x

In [0]:
model = CustomNet()

In [0]:
ce_loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
LEARNING_RATE = 0.0001
EPOCHS = 3
BATCH_SIZE = 16

In [0]:
optimizer = tf.optimizers.Adam(learning_rate= LEARNING_RATE)

In [0]:
def train_step(model, inputs, labels, loss_fn, optimzer):
    with tf.GradientTape() as t:
        y_predicted = model(inputs, training=True)
        current_loss = loss_fn(labels, y_predicted)
        
        gradients = t.gradient(current_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
    return current_loss

In [0]:
def valid_step(model, inputs, labels, loss_fn):
    y_predicted = model(inputs, training=False)
    current_loss = loss_fn(labels, y_predicted)
    return current_loss

In [0]:
train_dataset = train_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)

In [0]:
losses = tf.keras.metrics.Mean(name='loss')
val_losses = tf.keras.metrics.Mean(name='val_loss')

In [0]:
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = f'./temp/traincustom/{current_time}/logs'
file_writer = tf.summary.create_file_writer(train_log_dir)

In [0]:
# Checkpoint
# callbacks = [tf.keras.Checkpoint(...)]
checkpoint_dir = f'./temp/traincustom/checkpoints/'

In [0]:
ckpt = tf.train.Checkpoint(step = tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=3)

In [0]:
def check_for_checkpoint(manager):
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print(f"restored from {manager.latest_checkpoint}")
    else:
        print("Initializing from scratch")

In [76]:
check_for_checkpoint(manager)


restored from ./temp/traincustom/checkpoints/ckpt-10


In [77]:
for epoch in range(EPOCHS):

    print(f'epoch: {epoch}')
    losses.reset_states()
    val_losses.reset_states()
    for x_batch, y_batch in train_dataset:
        #print(x_batch)
        loss = train_step(model, x_batch, y_batch, ce_loss, optimizer)
        losses(loss)
#         step += 1
    
    save_path = manager.save()
    print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
    
    ckpt.step.assign_add(1)
    
    with file_writer.as_default():
        tf.summary.scalar('loss', losses.result(), step=epoch)
        tf.summary.image('Input images', x_batch, step=epoch)

    print(losses.result())
        
    for x_batch, y_batch in valid_dataset:
        val_loss = valid_step(model, x_batch, y_batch, ce_loss)
        val_losses(val_loss)
    
    with file_writer.as_default():
        tf.summary.scalar('val_loss', val_losses.result(), step=epoch)

epoch: 0
Saved checkpoint for step 7: ./temp/traincustom/checkpoints/ckpt-11
tf.Tensor(1.5121229, shape=(), dtype=float32)
epoch: 1
Saved checkpoint for step 8: ./temp/traincustom/checkpoints/ckpt-12
tf.Tensor(1.5087172, shape=(), dtype=float32)
epoch: 2


In [0]:
def predict(inputs):
    predicted = model(inputs)
    return tf.nn.softmax(predicted)

In [82]:
for sample, label in test_dataset.batch(16).take(4):
    predictions = predict(sample)
    print(tf.argmax(predictions,axis=1), label)
    #print(predictions)

tf.Tensor([4 4 9 7 5 1 0 5 7 4 0 8 2 3 9 0], shape=(16,), dtype=int64) tf.Tensor([4. 4. 9. 7. 5. 1. 0. 5. 7. 4. 0. 8. 2. 3. 9. 0.], shape=(16,), dtype=float32)
tf.Tensor([7 7 2 2 0 4 4 2 0 7 7 4 2 2 7 5], shape=(16,), dtype=int64) tf.Tensor([7. 7. 2. 2. 0. 4. 4. 4. 2. 7. 7. 4. 2. 4. 7. 5.], shape=(16,), dtype=float32)
tf.Tensor([9 5 4 4 3 3 1 7 5 3 0 0 0 6 9 9], shape=(16,), dtype=int64) tf.Tensor([9. 5. 4. 4. 3. 3. 1. 7. 5. 3. 0. 0. 0. 6. 9. 9.], shape=(16,), dtype=float32)
tf.Tensor([7 6 2 0 0 9 6 1 5 7 0 2 1 9 7 2], shape=(16,), dtype=int64) tf.Tensor([7. 6. 2. 0. 0. 9. 6. 1. 5. 7. 0. 2. 1. 9. 7. 2.], shape=(16,), dtype=float32)


In [0]:
model_dir = f'./temp/traincustom/models/'
weights_path = os.path.join(model_dir, 'weightsfashion.h5')

In [0]:
model.save_weights(weights_path)